In [2]:
from oil.lazy.lazy_matrix import LazyMatrix, lazy
from oil.lazy.low_rank import LowRank
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from oil.lazy.translation import translated_methods as np

In [4]:
import numpy as np

In [5]:
float_type = np.float64
## Toy Problem 2 for stochastic eigen decomposition
## PCA
mb = 1000
rank = 100
n = int(1e6)
toy_eigs2 = np.sqrt(.9*np.ones(rank))
toy_eigs2[0] = 1
toy_eigs2[:10] = np.array([1.1,.95,.7,.5,.3,.01,.001,-.4,-.7,-.94]).astype(np.float32)
S = np.diag(toy_eigs2)
#U = torch.ones(rank,1e6).normal_().numpy()
U = np.random.randn(n,rank).astype(float_type)
U,_ = np.linalg.qr(U)
V = np.random.randn(rank,rank).astype(float_type)
V,_ = np.linalg.qr(V)
#V = V.astype(np.float64)
X = (np.sqrt(n)*U@S@V.T).astype(float_type)
cov = X.T@X/n

index = toy_eigs2.argmax()
v = V.T[index]

def toy_A():
    indices = np.random.permutation(n).reshape(n//mb,mb)
    for batch_ids in indices:
        m = len(batch_ids)
        x = X[batch_ids]
        yield [lambda v: (1/m)*x.T@(x@v)]

In [5]:
m = 1000
rank = 10
n = int(1e5)
toy_eigs2 = np.sqrt(.9*np.ones(rank))
toy_eigs2[0] = 1
toy_eigs2[:10] = np.array([1.1,.95,.7,.5,.3,.01,.001,-.4,-.7,-.94]).astype(np.float32)
S = np.diag(toy_eigs2)
#U = torch.ones(rank,1e6).normal_().numpy()
U = np.random.randn(n,rank).astype(float_type)
U,_ = np.linalg.qr(U)
V = np.random.randn(m,rank).astype(float_type)
V,_ = np.linalg.qr(V)
#V = V.astype(np.float64)
X = (np.sqrt(n)*U@S@V.T).astype(float_type)

In [6]:
X.shape

(100000, 1000)

In [7]:
U.shape

(100000, 10)

In [8]:
V.shape

(1000, 10)

In [9]:
X_lazy = LowRank(U@S,V)

In [10]:
X

array([[-0.0201443 ,  0.08423717, -0.06563493, ..., -0.01030941,
         0.03612956, -0.0635186 ],
       [ 0.03995929,  0.09286144,  0.00573715, ...,  0.05376216,
         0.01175243, -0.02000221],
       [ 0.05157919,  0.0364231 ,  0.01054366, ...,  0.04015984,
        -0.00526227,  0.03573965],
       ...,
       [-0.07109255,  0.0080876 ,  0.03063208, ...,  0.00093946,
        -0.01276146,  0.00606065],
       [-0.01300325,  0.08528001, -0.0320387 , ..., -0.00859781,
        -0.04326792,  0.03297083],
       [ 0.03808794, -0.10250103, -0.02123413, ..., -0.06161583,
         0.0153169 ,  0.03055144]])

In [11]:
X_lazy.evaluate()

KeyError: 'index'

In [12]:
U@S@V.T

array([[-6.37018596e-05,  2.66381327e-04, -2.07555878e-04, ...,
        -3.26012196e-05,  1.14251692e-04, -2.00863437e-04],
       [ 1.26362383e-04,  2.93653648e-04,  1.81424680e-05, ...,
         1.70010879e-04,  3.71644327e-05, -6.32525402e-05],
       [ 1.63107711e-04,  1.15179954e-04,  3.33419708e-05, ...,
         1.26996554e-04, -1.66407469e-05,  1.13018696e-04],
       ...,
       [-2.24814369e-04,  2.55752272e-05,  9.68671429e-05, ...,
         2.97082692e-06, -4.03552746e-05,  1.91654484e-05],
       [-4.11198805e-05,  2.69679069e-04, -1.01315260e-04, ...,
        -2.71886475e-05, -1.36825190e-04,  1.04262923e-04],
       [ 1.20444637e-04, -3.24136704e-04, -6.71482004e-05, ...,
        -1.94846352e-04,  4.84362831e-05,  9.66121446e-05]])

In [13]:
lazy(U)

In [14]:
X_lazy2 = lazy(U)@lazy(S)@lazy(V.T)

In [ ]:
X_lazy2.evaluate()

In [ ]:
np.eye(10)

In [12]:
np.arange(10,1).shape

(0,)

In [15]:
np.ones((10,1)).shape*2

(10, 1, 10, 1)